In [ ]:
# ---------------------------------------------------------
# BB84 Quantum Key Distribution Simulation in Qiskit 2.x
# Generates a 3D Surface Plot of QBER vs P_Eve and P_Noise
# ---------------------------------------------------------

from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Required for 3D plotting

# --- Simulation Parameters ---
# NOTE: N_QUBITS is reduced for the sweep to keep computation time feasible
N_QUBITS = 10000
SECURITY_THRESHOLD = 0.11  # QBER limit (11%) for secure key generation

# --- Sweep Parameters ---
P_EVE_SWEEP_POINTS = 10
P_NOISE_SWEEP_POINTS = 10
P_EVE_VALUES = np.linspace(0.0, 1.0, P_EVE_SWEEP_POINTS)
P_NOISE_VALUES = np.linspace(0.0, 0.10, P_NOISE_SWEEP_POINTS)

# ---------------------------------------------------------
# Helper Functions (Unchanged from previous versions)
# ---------------------------------------------------------

def prepare_qubit(bit, basis):
    """ Prepare a single BB84 qubit. """
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)
    if basis == 'X':
        qc.h(0)
    return qc

def measure_qubit(qc, basis):
    """ Measure a qubit in a given basis. """
    if basis == 'X':
        qc.h(0)
    qc.measure(0, 0)

    backend = Aer.get_backend("aer_simulator")
    job = backend.run(qc, shots=1)
    result = job.result()
    counts = result.get_counts()
    measured_bit_str = max(counts, key=counts.get) if counts else '0'
    return int(measured_bit_str)

def eve_intercept(bit, basis, eve_prob):
    """ Simulates Eve intercept-resend with probability eve_prob. """
    if random.random() > eve_prob:
        return None  # Eve does nothing

    eve_basis = random.choice(['Z', 'X'])
    eve_qc = prepare_qubit(bit, basis)
    eve_bit = measure_qubit(eve_qc, eve_basis)

    resend_qc = prepare_qubit(eve_bit, eve_basis)
    return resend_qc

def make_noise_model(p=0.0):
    """ Simple depolarizing noise model for BB84. p=0 for ideal. """
    if p == 0:
        return None
    noise_model = NoiseModel()
    error = depolarizing_error(p, 1)
    noise_model.add_all_qubit_quantum_error(error, ['x', 'h'])
    return noise_model

# ---------------------------------------------------------
# Main BB84 Simulation (Unchanged from previous versions)
# ---------------------------------------------------------

def run_bb84(N, eve_prob, noise_model=None):
    """
    Run BB84 simulation.
    """
    backend = Aer.get_backend("aer_simulator")

    alice_bits = np.random.randint(2, size=N)
    alice_bases = np.random.choice(['Z', 'X'], size=N)
    bob_bases = np.random.choice(['Z', 'X'], size=N)

    bob_results = []

    for i in range(N):
        bit = alice_bits[i]
        a_basis = alice_bases[i]
        b_basis = bob_bases[i]

        qc = prepare_qubit(bit, a_basis)
        eve_qc = eve_intercept(bit, a_basis, eve_prob)
        if eve_qc is not None:
            qc = eve_qc

        meas_qc = qc.copy()
        if b_basis == 'X':
            meas_qc.h(0)
        meas_qc.measure(0, 0)

        job = backend.run(meas_qc, shots=1, noise_model=noise_model)
        result = job.result()
        counts = result.get_counts()
        
        measured_bit_str = max(counts, key=counts.get) if counts else '0'
        measured_bit = int(measured_bit_str)
        bob_results.append(measured_bit)

    mask = alice_bases == bob_bases
    sifted_alice = alice_bits[mask]
    sifted_bob = np.array(bob_results)[mask]

    sifted_len = len(sifted_alice)
    if sifted_len == 0:
        QBER = 0.0
    else:
        errors = np.sum(sifted_alice != sifted_bob)
        QBER = errors / sifted_len

    return QBER, sifted_len


# ---------------------------------------------------------
# NEW Plotting Function: 3D Surface Plot (Fix applied)
# ---------------------------------------------------------

def run_and_plot_3d_sweep(n_qubits, p_eve_values, p_noise_values):
    """
    Runs a 2D sweep over P_Eve and P_Noise and generates a 3D surface plot of QBER.
    """
    # Create meshgrid for plotting
    X, Y = np.meshgrid(p_eve_values, p_noise_values)
    Z = np.zeros(X.shape)
    
    # Store QBER values in the Z matrix
    for i, p_noise in enumerate(p_noise_values):
        noise_model = make_noise_model(p=p_noise)
        for j, p_eve in enumerate(p_eve_values):
            # Run simulation (average 2 trials for faster generation)
            trials = 2
            avg_qber = np.mean([run_bb84(n_qubits, p_eve, noise_model)[0] for _ in range(trials)])
            Z[i, j] = avg_qber

    # --- 3D Plotting ---
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the surface
    surface = ax.plot_surface(X, Y, Z, 
                              cmap='viridis', 
                              edgecolor='none', 
                              alpha=0.8)

    # --- Security Threshold Plane ---
    # Create a plane at Z = SECURITY_THRESHOLD
    X_thresh, Y_thresh = np.meshgrid(p_eve_values, p_noise_values)
    Z_thresh = np.full_like(X_thresh, SECURITY_THRESHOLD)
    
    # Plot the threshold plane with a semi-transparent red color
    # NOTE: Removed 'label' argument here to fix the AttributeError
    ax.plot_surface(X_thresh, Y_thresh, Z_thresh, 
                    color='red', 
                    alpha=0.3)
    
    # --- Formatting ---
    fig.colorbar(surface, shrink=0.5, aspect=5, label='QBER (Z-Axis)')
    
    ax.set_title(f'BB84 Security Phase Space (N={n_qubits})', fontsize=16)
    ax.set_xlabel('Eavesdropping Probability ($P_{Eve}$)', fontsize=12)
    ax.set_ylabel('Depolarizing Noise Strength ($P_{Noise}$)', fontsize=12)
    ax.set_zlabel('Quantum Bit Error Rate (QBER)', fontsize=12)

    # Add a legend for the threshold plane using a dummy line artist
    ax.plot([], [], color='red', alpha=0.5, label=f'Security Threshold ({SECURITY_THRESHOLD*100:.0f}%)')
    ax.legend()
    
    plt.show()

    return Z


# ---------------------------------------------------------
# Run All Experiments
# ---------------------------------------------------------

if __name__ == "__main__":
    print(f"--- BB84 3D Surface Plot Setup ---")
    print(f"Key Length per sweep point (N): {N_QUBITS}")
    print(f"P_Eve Sweep Points: {P_EVE_SWEEP_POINTS} ({P_EVE_VALUES[0]:.2f} to {P_EVE_VALUES[-1]:.2f})")
    print(f"P_Noise Sweep Points: {P_NOISE_SWEEP_POINTS} ({P_NOISE_VALUES[0]:.2f} to {P_NOISE_VALUES[-1]:.2f})")
    print(f"Total Simulations: {P_EVE_SWEEP_POINTS * P_NOISE_SWEEP_POINTS}")
    print(f"Security Threshold Plane: {SECURITY_THRESHOLD*100:.0f}%")

    print("\nRunning 2D sweep to generate 3D QBER surface...")
    
    # Generate the 3D Figure
    qber_surface_data = run_and_plot_3d_sweep(N_QUBITS, P_EVE_VALUES, P_NOISE_VALUES)
    
    print("\nSimulation complete. The 3D surface plot is generated.")
    print("The plot shows the relationship QBER = f($P_{Eve}, P_{Noise}$).")
    print("The red plane visually represents the point at which key generation should be aborted.")

In [ ]:
# ---------------------------------------------------------
# Code to Redraw the 3D Plot from Saved Simulation Data
# This code assumes 'qber_surface_data' (the Z matrix) exists
# in your environment from the previous run.
# ---------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- Redefine Constants and Axes from Previous Run ---
# IMPORTANT: These must match the settings used in the simulation
SECURITY_THRESHOLD = 0.11
P_EVE_SWEEP_POINTS = 10
P_NOISE_SWEEP_POINTS = 10
P_EVE_VALUES = np.linspace(0.0, 1.0, P_EVE_SWEEP_POINTS)
# Note: Using 0.10 for noise maximum, as per your last file's parameters
P_NOISE_VALUES = np.linspace(0.0, 0.10, P_NOISE_SWEEP_POINTS)

# Recreate the meshgrid (X and Y coordinates)
X, Y = np.meshgrid(P_EVE_VALUES, P_NOISE_VALUES)

# You must replace Z with your saved data variable name:
Z = qber_surface_data 


def redraw_3d_plot(X, Y, Z, security_threshold, elev, azim, title_suffix=""):
    """
    Redraws the 3D surface plot from existing X, Y, Z data at a new angle.

    elev: vertical viewing angle (e.g., 20)
    azim: horizontal rotation angle (e.g., -140)
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Set the new viewing angle
    ax.view_init(elev=elev, azim=azim)
    
    # Plot the QBER surface
    surface = ax.plot_surface(X, Y, Z, 
                              cmap='viridis', 
                              edgecolor='none', 
                              alpha=0.8)

    # --- Security Threshold Plane ---
    X_thresh, Y_thresh = np.meshgrid(X[0,:], Y[:,0])
    Z_thresh = np.full_like(X_thresh, security_threshold)
    
    ax.plot_surface(X_thresh, Y_thresh, Z_thresh, 
                    color='red', 
                    alpha=0.3)
    
    # --- Formatting ---
    fig.colorbar(surface, shrink=0.5, aspect=5, label='QBER (Z-Axis)')
    
    ax.set_title(f'BB84 Security Phase Space - {title_suffix}', fontsize=16)
    ax.set_xlabel('Eavesdropping Probability ($P_{Eve}$)', fontsize=12)
    ax.set_ylabel('Depolarizing Noise Strength ($P_{Noise}$)', fontsize=12)
    ax.set_zlabel('Quantum Bit Error Rate (QBER)', fontsize=12)

    # Add a legend for the threshold plane
    ax.plot([], [], color='red', alpha=0.5, label=f'Security Threshold ({security_threshold*100:.0f}%)')
    ax.legend()
    
    plt.show()


# ---------------------------------------------------------
# Execute Redraws
# ---------------------------------------------------------

print("\nRedrawing 3D plot from a new angle (Side View)...")

# New Angle 1: Side-View (Good for showing the intersection)
redraw_3d_plot(X, Y, Z, SECURITY_THRESHOLD, elev=25, azim=-140, title_suffix="Side View (E25, A-140)")

print("\nRedrawing 3D plot from a third angle (Top-Down View)...")

# New Angle 2: Top-Down View (Good for showing the shape of the function)
redraw_3d_plot(X, Y, Z, SECURITY_THRESHOLD, elev=55, azim=-90, title_suffix="Top-Down View (E55, A-90)")

print("\nTwo new views of the QBER surface have been generated without re-simulation!")

In [ ]:
# ---------------------------------------------------------
# QBER Security Boundary Plot (2D Contour)
# Plots the line where QBER = 11% (The Secure/Insecure boundary)
# This code assumes 'qber_surface_data' (Z matrix) exists
# in your environment from the previous 3D run.
# ---------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt

# --- Redefine Constants and Axes from Previous Run ---
# IMPORTANT: These must match the settings used in the simulation
SECURITY_THRESHOLD = 0.11
P_EVE_SWEEP_POINTS = 10
P_NOISE_SWEEP_POINTS = 10
P_EVE_VALUES = np.linspace(0.0, 1.0, P_EVE_SWEEP_POINTS)
# Note: Using 0.10 for noise maximum, as per your last file's parameters
P_NOISE_VALUES = np.linspace(0.0, 0.10, P_NOISE_SWEEP_POINTS)

# You must replace Z with your saved data variable name:
# This assumes the variable containing the QBER data is named 'qber_surface_data'
Z = qber_surface_data 


def plot_security_boundary(p_noise_values, p_eve_values, qber_data, threshold):
    """
    Generates a 2D contour plot of the QBER = 0.11 boundary.
    """
    fig, ax = plt.subplots(figsize=(8, 6))

    # --- Plotting the Contour ---
    # We plot P_Noise on X, P_Eve on Y, and use the transposed QBER data (Z.T)
    # to match the axis orientation requested.
    contour = ax.contour(
        p_noise_values,  # X-axis (P_Noise)
        p_eve_values,  # Y-axis (P_Eve)
        qber_data.T,  # Z data (QBER), transposed to align axes
        levels=[threshold],  # Find the exact 0.11 level
        colors='red',
        linestyles='-',
        linewidths=3,
        label='Security Boundary'
    )
    
    # Fill the 'secure' region (below 0.11) in green
    ax.contourf(
        p_noise_values,
        p_eve_values,
        qber_data.T,
        levels=[0.0, threshold],
        colors=['#CCFFCC'], # Light green for secure zone
        alpha=0.6
    )

    # --- Formatting ---
    
    # Add labels to the contour line itself (optional but helpful)
    ax.clabel(contour, inline=True, fontsize=10, fmt={threshold: f'QBER = {threshold*100:.0f}%'})

    ax.set_title('BB84 Security Boundary: Maximum Tolerable Eavesdropping', fontsize=14)
    ax.set_xlabel('Channel Noise Strength ($P_{Noise}$)', fontsize=12)
    ax.set_ylabel('Maximum Eavesdropping Probability ($P_{Eve}$)', fontsize=12)
    
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Highlight the theoretical point for an ideal channel
    # (P_Noise=0, P_Eve=0.44)
    ax.plot(0, threshold/0.25, marker='*', markersize=12, color='blue', 
            label=f'Ideal Max $P_{{Eve}}$ ({threshold/0.25:.2f})')
    
    # Add shading for the insecure zone (above the threshold)
    ax.plot([0, p_noise_values[-1]], [threshold/0.25, threshold/0.25], 
            'k--', alpha=0.2, linewidth=1)
    
    # Add explanatory text for the insecure zone
    ax.text(p_noise_values[-1] * 0.7, p_eve_values[-1] * 0.9, 
            'INSECURE ZONE', color='darkred', fontsize=12, fontweight='bold')
    ax.text(p_noise_values[0] * 1.5, p_eve_values[0] * 1.5, 
            'SECURE ZONE', color='darkgreen', fontsize=12, fontweight='bold')
    
    ax.legend(loc='lower left')
    plt.show()


# ---------------------------------------------------------
# Execute Plotting
# ---------------------------------------------------------

print("\nGenerating 2D Security Boundary Contour Plot...")

plot_security_boundary(P_NOISE_VALUES, P_EVE_VALUES, Z, SECURITY_THRESHOLD)

print("\nContour plot generated. This figure explicitly shows the maximum P_Eve allowed for any given P_Noise.")